In [1]:
import pandas as pd
import geopandas as gpd
import io
import json
import os

os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
metadata_df = pd.read_csv("output_metadata.csv")

In [2]:
from src.download import DataDownloaderUploader
from src.process import Processor
from src.document import DataDocumenter
from src.export import s3Exporter

In [3]:
DataDownloaderUploader().download_and_upload_main()
Processor().process_raw_datasets()
DataDocumenter().document_processed_data()
s3Exporter().export_all("export_test")

117it [00:02, 42.17it/s]
  0%|          | 0/100 [00:00<?, ?it/s]

Processing data/raw/administrative/blm/grsg_biologically_significant_units.geojson
Skipping data/raw/administrative/blm/grsg_biologically_significant_units.geojson, which has already been processed.
Processing data/raw/administrative/blm/grsg_habitat_management_areas.geojson
Skipping data/raw/administrative/blm/grsg_habitat_management_areas.geojson, which has already been processed.
Processing data/raw/administrative/blm/wilderness_study_areas.geojson
Skipping data/raw/administrative/blm/wilderness_study_areas.geojson, which has already been processed.
Processing data/raw/administrative/boundaries_and_management/wind_river_reservation.geojson
Skipping data/raw/administrative/boundaries_and_management/wind_river_reservation.geojson, which has already been processed.
Processing data/raw/administrative/us_fish_and_wildlife/refuge_boundaries.geojson
Skipping data/raw/administrative/us_fish_and_wildlife/refuge_boundaries.geojson, which has already been processed.
Processing data/raw/adminis

 10%|█         | 10/100 [00:14<02:08,  1.43s/it]

Error processing data/raw/climate_change_indicators/flooding/fema_nfhl.zip: '/vsizip//vsimem/45335e8cbeb64bd68f3af09011f56960.zip' not recognized as a supported file format.
Processing data/raw/crucial_critical/blm/areas_of_critical_environmental_concern.zip


100%|██████████| 100/100 [00:15<00:00,  6.48it/s]

Error processing data/raw/crucial_critical/blm/areas_of_critical_environmental_concern.zip: '/vsizip//vsimem/048cd22389174f29b1f62608e7b4d0fd.zip' not recognized as a supported file format.
Processing data/raw/crucial_critical/crucial_range/antelope.geojson
Skipping data/raw/crucial_critical/crucial_range/antelope.geojson, which has already been processed.
Processing data/raw/crucial_critical/crucial_range/bighorn_sheep.geojson
Skipping data/raw/crucial_critical/crucial_range/bighorn_sheep.geojson, which has already been processed.
Processing data/raw/crucial_critical/crucial_range/elk.geojson
Skipping data/raw/crucial_critical/crucial_range/elk.geojson, which has already been processed.
Processing data/raw/crucial_critical/crucial_range/moose.geojson
Skipping data/raw/crucial_critical/crucial_range/moose.geojson, which has already been processed.
Processing data/raw/crucial_critical/crucial_range/mule_deer.geojson
Skipping data/raw/crucial_critical/crucial_range/mule_deer.geojson, whi

data/processed/administrative/blm/grsg_biologically_significant_units.geojson
data/processed/administrative/blm/grsg_habitat_management_areas.geojson
data/processed/administrative/blm/wilderness_study_areas.geojson
data/processed/administrative/boundaries_and_management/wind_river_reservation.geojson
data/processed/administrative/us_fish_and_wildlife/refuge_boundaries.geojson
data/processed/administrative/us_forest_service/national_forests.geojson
data/processed/administrative/us_forest_service/national_grassland_units.geojson
data/processed/administrative/us_forest_service/priority_watersheds.geojson
data/processed/administrative/us_forest_service/special_interest_management_areas.geojson
data/processed/crucial_critical/crucial_range/antelope.geojson
data/processed/crucial_critical/crucial_range/bighorn_sheep.geojson
data/processed/crucial_critical/crucial_range/elk.geojson
data/processed/crucial_critical/crucial_range/moose.geojson
data/processed/crucial_critical/crucial_range/mule_d

In [ ]:
gdf = gpd.read_file("wyoming.geojson")

In [ ]:
gpd.read_file("wyoming.geojson").to_crs(32045).area


In [ ]:
gpd.read_file("wyoming.geojson").geometry.geom_type.value_counts()


In [ ]:
gdf.to_crs(32045).area.sum() / 60

In [ ]:
gpd.read_file("/Users/canyonfoot/Downloads/Data/FPA_FOD_20221014.gpkg")